In [1]:
from astropy.table import Table

In [2]:
GASC = Table.read("GALEX-GASC_20170116.fits")
GMSC =  Table.read("GALEX-GMSC_20170116.fits")

## Column to keep

- name: Unique identifier
- ra, dec
- field: Column added to the pristine catalogue to contain the HELP field name.
- flux_nuv, fluxerr_nuv, flux_fuv, fluxerr_nuv: Total fluxes en errors

## Flag selection

EXTENDED: deblending parameters in the SExtractor program correctly measure most galaxies up to one arcminute in diameter. Galaxies larger than this will have a larger probability of being shredded into multiple sources by the GALEX pipeline. So that users can avoid these shredded galaxies in the catalog,  the "EXTENDED" flag is given in the catalog for known optical sources with diameters (typically 25th magnitude isophote optical diameters, D25) greater than 1.5 arcminute. If a source in the catalogs lies within an elliptical aperture with major axis scaled to 1.25*D25, then EXTENDED is equal to one and zero otherwise.

—> I propose to drop objects with EXTENDED equal to 1

ARTIFACT_NUV, ARTIFACT_FUV: it is preconised to exclude sources with flags 2 and 3. It is also safer to exclude measures close to the edge of the field (flag=6)

—> exclude objects with ARTIFACT=2 or 3 or 6

MANFLAG: There are additional artifacts due to bright stars lying just off of the field-of-view that are not in general flagged by the standard GALEX pipeline. These artifacts are in general shaped like horseshoes or long, thin cones. The primary regions of both the GASC and GMSC were visually inspected and these artifacts are identified. For any source that lies within one of these flagged regions, the column MANFLAG will be set to one. In addition bright stars  can create thin elliptical sources somewhat resembling an edge-on galaxy that lie perpendicular to the edge of the field-of-view. The location and shape parameters in the catalogs are used to flag such sources. Any source that is likely one of these artifacts will have MANFLAG set equal to two. 

—> Use only sources with MANFLAG=0

### GASC mask

In [3]:
gasc_extended_mask = GASC['extended'] != 1
gasc_artefact_nuv_mask = (GASC['artefact_nuv'] != 2) & (GASC['artefact_nuv'] != 3) & \
    (GASC['artefact_nuv'] != 6)
gasc_artefact_fuv_mask = (GASC['artefact_fuv'] != 2) & (GASC['artefact_fuv'] != 3) & \
    (GASC['artefact_fuv'] != 6)
gasc_manflag_mask = GASC['manflag'] == 0
gasc_mask = gasc_extended_mask & gasc_artefact_nuv_mask & gasc_artefact_fuv_mask & \
    gasc_manflag_mask

### GMSC mask

In [4]:
gmsc_extended_mask = GMSC['extended'] != 1
gmsc_artefact_nuv_mask = (GMSC['artefact_nuv'] != 2) & (GMSC['artefact_nuv'] != 3) & \
    (GMSC['artefact_nuv'] != 6)
gmsc_artefact_fuv_mask = (GMSC['artefact_fuv'] != 2) & (GMSC['artefact_fuv'] != 3) & \
    (GMSC['artefact_fuv'] != 6)
gmsc_manflag_mask = GMSC['manflag'] == 0
gmsc_mask = gmsc_extended_mask & gmsc_artefact_nuv_mask & gmsc_artefact_fuv_mask & \
    gmsc_manflag_mask

## Creation of the HELP catalogue

In [5]:
help_gasc = GASC[
    'field', 'name', 'ra', 'dec', 'flux_nuv', 'fluxerr_nuv', 'flux_fuv', 'fluxerr_fuv'
][gasc_mask]

In [6]:
help_gmsc = GMSC[
    'field', 'name', 'ra', 'dec', 'flux_nuv', 'fluxerr_nuv', 'flux_fuv', 'fluxerr_fuv'
][gmsc_mask]

## Notes

The fluxes are already in μJy. They are not corrected for galactic extinction.  This correction will occur during datafusion to use the same E(B-V) as for other fluxes to be corrected.  The flux columns will then be named F_NUV, FErr_NUV, F_FUV, FErr_NUV to be usable by CIGALE.

## Saving the catalogues

In [7]:
help_gasc.write("GALEX-GASC_20170116_HELP-SELECTED.fits")
help_gmsc.write("GALEX-GMSC_20170116_HELP-SELECTED.fits")

## Some check

In [8]:
print("The flag selection in the GASC catalogue changed the number of sources"
     "from {} to {}".format(len(GASC), len(help_gasc)))
print("The flag selection in the GMSC catalogue changed the number of sources"
     "from {} to {}".format(len(GMSC), len(help_gmsc)))

The flag selection in the GASC catalogue changed the number of sourcesfrom 1924072 to 1884348
The flag selection in the GMSC catalogue changed the number of sourcesfrom 3163240 to 3121917
